<a href="https://colab.research.google.com/github/devaharsha908/AI-Based-Cyber-Security-Threat-Prediction/blob/main/SQL_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sqlite3
import pandas as pd

In [4]:
conn = sqlite3.connect("ride_analysis.db")
cursor = conn.cursor()

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS ncr_ride_bookings (
    date TEXT,
    time TEXT,
    booking_id TEXT PRIMARY KEY,
    booking_status TEXT,
    customer_id TEXT,
    vehicle_type TEXT,
    pickup_location TEXT,
    drop_location TEXT,
    avg_vtat REAL,
    avg_ctat REAL,
    cancelled_rides_by_customer INTEGER,
    reason_for_cancelling_by_customer TEXT,
    cancelled_rides_by_driver INTEGER,
    driver_cancellation_reason TEXT,
    incomplete_rides INTEGER,
    incomplete_rides_reason TEXT,
    booking_value REAL,
    ride_distance REAL,
    driver_ratings REAL,
    customer_rating REAL,
    payment_method TEXT,
    pickup_datetime TEXT
)
""")

In [7]:
from google.colab import files

uploaded = files.upload()



Saving archive.zip to archive.zip


In [9]:
import zipfile
import os

# Extract the uploaded archive.zip
with zipfile.ZipFile("archive.zip", "r") as z:
    z.extractall("data")

# List files inside extracted folder
os.listdir("data")


['Dasboard.gif', 'ncr_ride_bookings.csv', 'Uber.pbix']

In [12]:
import os
os.listdir()


['.config', 'archive.zip', 'ride_analysis.db', 'data', 'sample_data']

In [14]:
import zipfile
import os

# Unzip archive.zip into current directory
with zipfile.ZipFile("archive.zip", "r") as zip_ref:
    zip_ref.extractall(".")

# Check extracted files
os.listdir()


['.config',
 'Dasboard.gif',
 'archive.zip',
 'ride_analysis.db',
 'data',
 'ncr_ride_bookings.csv',
 'Uber.pbix',
 'sample_data']

In [15]:
import pandas as pd

df = pd.read_csv("ncr_ride_bookings.csv")
df.head()


,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


In [16]:
import sqlite3

# Connect to SQLite
conn = sqlite3.connect("ride_analysis.db")

# Insert into SQL table
df.to_sql("ncr_ride_bookings", conn, if_exists="replace", index=False)

# Quick test
pd.read_sql("SELECT COUNT(*) as total_rides FROM ncr_ride_bookings;", conn)


,total_rides
0,150000


In [17]:
df = pd.read_csv("ncr_ride_bookings.csv")

In [18]:
df.to_sql("ncr_ride_bookings", conn, if_exists="replace", index=False)

150000

In [19]:
query = "SELECT COUNT(*) as total_rides FROM ncr_ride_bookings;"
result = pd.read_sql(query, conn)
print(result)

   total_rides
0       150000


**Prompt 1: Basic dataset overview**

In [23]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect("ride_analysis.db")

# Run SQL inside pd.read_sql
query = """
SELECT
    COUNT(*) AS total_rows,
    COUNT(DISTINCT "Customer ID") AS unique_customers
FROM ncr_ride_bookings;
"""
result = pd.read_sql(query, conn)
print(result)


   total_rows  unique_customers
0      150000            148788


**Prompt 2: Check missing values per column**

In [25]:
import pandas as pd

query_missing = """
SELECT
    SUM(CASE WHEN "Ride Distance" IS NULL THEN 1 ELSE 0 END) AS missing_ride_distance,
    SUM(CASE WHEN "Booking Value" IS NULL THEN 1 ELSE 0 END) AS missing_booking_value,
    SUM(CASE WHEN "Driver Ratings" IS NULL THEN 1 ELSE 0 END) AS missing_driver_ratings,
    SUM(CASE WHEN "Customer Rating" IS NULL THEN 1 ELSE 0 END) AS missing_customer_rating
FROM ncr_ride_bookings;
"""

df_missing = pd.read_sql(query_missing, conn)
display(df_missing)



,missing_ride_distance,missing_booking_value,missing_driver_ratings,missing_customer_rating
0,48000,48000,57000,57000


**Prompt 3: Distribution of ride_distance (bucketed)**

In [26]:
query = """
SELECT
    FLOOR("Ride Distance") AS distance_bucket,
    COUNT(*) AS ride_count
FROM ncr_ride_bookings
WHERE "Ride Distance" IS NOT NULL
GROUP BY FLOOR("Ride Distance")
ORDER BY distance_bucket;
"""
pd.read_sql(query, conn)


,distance_bucket,ride_count
0,1.0,481
1,2.0,2342
2,3.0,2336
3,4.0,2419
4,5.0,2382
5,6.0,2406
6,7.0,2464
7,8.0,2477
8,9.0,2434
9,10.0,2355


**Prompt 4: Average ride_distance per vehicle_type**

In [27]:
query = """
SELECT "Vehicle Type", AVG("Ride Distance") AS avg_distance
FROM ncr_ride_bookings
GROUP BY "Vehicle Type";
"""
pd.read_sql(query, conn)


,Vehicle Type,avg_distance
0,Auto,24.615996
1,Bike,24.649546
2,Go Mini,24.612090
3,Go Sedan,24.609325
4,Premier Sedan,24.598956
5,Uber XL,24.402893
6,eBike,24.990415


**Prompt 5: Booking_status counts**

In [28]:
query = """
SELECT "Booking Status", COUNT(*) AS total
FROM ncr_ride_bookings
GROUP BY "Booking Status";
"""
pd.read_sql(query, conn)


,Booking Status,total
0,Cancelled by Customer,10500
1,Cancelled by Driver,27000
2,Completed,93000
3,Incomplete,9000
4,No Driver Found,10500


**Prompt 6: Top 10 pickup locations**

In [29]:
query = """
SELECT "Pickup Location", COUNT(*) AS total
FROM ncr_ride_bookings
GROUP BY "Pickup Location"
ORDER BY total DESC
LIMIT 10;
"""
pd.read_sql(query, conn)


,Pickup Location,total
0,Khandsa,949
1,Barakhamba Road,946
2,Saket,931
3,Badarpur,921
4,Pragati Maidan,920
5,Madipur,919
6,AIIMS,918
7,Mehrauli,915
8,Dwarka Sector 21,914
9,Pataudi Chowk,907


**Prompt 7: Top 10 drop locations**

In [30]:
query = """
SELECT "Drop Location", COUNT(*) AS total
FROM ncr_ride_bookings
GROUP BY "Drop Location"
ORDER BY total DESC
LIMIT 10;
"""
pd.read_sql(query, conn)


,Drop Location,total
0,Ashram,936
1,Basai Dhankot,917
2,Lok Kalyan Marg,916
3,Narsinghpur,913
4,Kalkaji,912
5,Cyber Hub,912
6,Kashmere Gate ISBT,909
7,Udyog Vihar,906
8,Lajpat Nagar,904
9,Nehru Place,902


**Prompt 8: Cancelled rides by customer vs driver**

In [31]:
query = """
SELECT
    SUM("Cancelled Rides by Customer") AS cancelled_by_customer,
    SUM("Cancelled Rides by Driver") AS cancelled_by_driver
FROM ncr_ride_bookings;
"""
pd.read_sql(query, conn)


,cancelled_by_customer,cancelled_by_driver
0,10500.0,27000.0


**Prompt 9: Most common customer cancellation reasons**

In [32]:
query = """
SELECT "Reason for cancelling by Customer", COUNT(*) AS total
FROM ncr_ride_bookings
WHERE "Reason for cancelling by Customer" IS NOT NULL
GROUP BY "Reason for cancelling by Customer"
ORDER BY total DESC
LIMIT 10;
"""
pd.read_sql(query, conn)


,Reason for cancelling by Customer,total
0,Wrong Address,2362
1,Change of plans,2353
2,Driver is not moving towards pickup location,2335
3,Driver asked to cancel,2295
4,AC is not working,1155


**Prompt 10: Most common driver cancellation reasons**

In [33]:
query = """
SELECT "Driver Cancellation Reason", COUNT(*) AS total
FROM ncr_ride_bookings
WHERE "Driver Cancellation Reason" IS NOT NULL
GROUP BY "Driver Cancellation Reason"
ORDER BY total DESC
LIMIT 10;
"""
pd.read_sql(query, conn)


,Driver Cancellation Reason,total
0,Customer related issue,6837
1,The customer was coughing/sick,6751
2,Personal & Car related issues,6726
3,More than permitted people in there,6686


***Prompt 11: Total booking_value per payment_method***

In [34]:
query = """
SELECT "Payment Method", SUM("Booking Value") AS total_value
FROM ncr_ride_bookings
GROUP BY "Payment Method";
"""
pd.read_sql(query, conn)


,Payment Method,total_value
0,None,NaN
1,Cash,12895649.0
2,Credit Card,5224025.0
3,Debit Card,4180510.0
4,UPI,23345101.0
5,Uber Wallet,6200898.0


**Prompt 12: Average driver_ratings by vehicle_type**

In [35]:
query = """
SELECT "Vehicle Type", AVG("Driver Ratings") AS avg_driver_rating
FROM ncr_ride_bookings
GROUP BY "Vehicle Type";
"""
pd.read_sql(query, conn)


,Vehicle Type,avg_driver_rating
0,Auto,4.232369
1,Bike,4.230056
2,Go Mini,4.227694
3,Go Sedan,4.231812
4,Premier Sedan,4.234865
5,Uber XL,4.238340
6,eBike,4.225614


**Prompt 13: Average customer_rating by vehicle_type**

In [36]:
query = """
SELECT "Vehicle Type", AVG("Customer Rating") AS avg_customer_rating
FROM ncr_ride_bookings
GROUP BY "Vehicle Type";
"""
pd.read_sql(query, conn)


,Vehicle Type,avg_customer_rating
0,Auto,4.402000
1,Bike,4.403940
2,Go Mini,4.404297
3,Go Sedan,4.409996
4,Premier Sedan,4.403457
5,Uber XL,4.404851
6,eBike,4.403954


**Prompt 14: Export ride_distance vs booking_value for correlation**

In [37]:
query = """
SELECT "Ride Distance", "Booking Value"
FROM ncr_ride_bookings
WHERE "Ride Distance" IS NOT NULL AND "Booking Value" IS NOT NULL;
"""
df_corr = pd.read_sql(query, conn)
df_corr.head()


,Ride Distance,Booking Value
0,5.73,237.0
1,13.58,627.0
2,34.02,416.0
3,48.21,737.0
4,4.85,316.0


**Prompt 15: Trend of rides per day**

In [38]:
query = """
SELECT DATE("Date") AS ride_date, COUNT("Booking ID") AS total_rides
FROM ncr_ride_bookings
GROUP BY DATE("Date")
ORDER BY ride_date;
"""
pd.read_sql(query, conn)


,ride_date,total_rides
0,2024-01-01,414
1,2024-01-02,389
2,2024-01-03,384
3,2024-01-04,414
4,2024-01-05,416
...,...,...
360,2024-12-26,369
361,2024-12-27,431
362,2024-12-28,391
363,2024-12-29,417


**Prompt 16: Peak booking times (hourly)**

In [39]:
query = """
SELECT strftime('%H', "pickup_datetime") AS hour, COUNT(*) AS total
FROM ncr_ride_bookings
GROUP BY strftime('%H', "pickup_datetime")
ORDER BY hour;
"""
pd.read_sql(query, conn)


,hour,total
0,None,150000


**Prompt 17: Incomplete rides and their reasons**

In [40]:
query = """
SELECT "Incomplete Rides Reason", COUNT(*) AS total
FROM ncr_ride_bookings
WHERE "Incomplete Rides Reason" IS NOT NULL
GROUP BY "Incomplete Rides Reason"
ORDER BY total DESC
LIMIT 10;
"""
pd.read_sql(query, conn)


,Incomplete Rides Reason,total
0,Customer Demand,3040
1,Vehicle Breakdown,3012
2,Other Issue,2948


**Prompt 18: Customer vs driver rating distribution**

In [41]:
query = """
SELECT "Driver Ratings", "Customer Rating", COUNT(*) AS total
FROM ncr_ride_bookings
GROUP BY "Driver Ratings", "Customer Rating"
ORDER BY total DESC;
"""
pd.read_sql(query, conn)


,Driver Ratings,Customer Rating,total
0,NaN,NaN,57000
1,4.2,4.9,1812
2,4.3,4.9,1786
3,4.2,4.6,1750
4,4.3,4.6,1725
...,...,...,...
437,3.2,3.5,5
438,3.0,3.0,4
439,3.0,3.3,4
440,3.5,3.0,4


**Prompt 19: High-value rides (top 5%) – handled in Python**

In [42]:
query = """
SELECT "Booking ID", "Booking Value", "Ride Distance"
FROM ncr_ride_bookings
WHERE "Booking Value" IS NOT NULL;
"""
df_values = pd.read_sql(query, conn)

# Compute 95th percentile in Python
cutoff = df_values["Booking Value"].quantile(0.95)
df_high_value = df_values[df_values["Booking Value"] >= cutoff].sort_values("Booking Value", ascending=False).head(20)
df_high_value


,Booking ID,Booking Value,Ride Distance
5096,"""CNR7954315""",4277.0,8.66
56743,"""CNR1798489""",4228.0,11.73
1873,"""CNR8487909""",4220.0,10.11
61507,"""CNR5182516""",4202.0,4.62
4870,"""CNR7356012""",4133.0,25.66
29602,"""CNR8849175""",4109.0,36.81
14310,"""CNR5553074""",4093.0,20.85
22438,"""CNR8715944""",4088.0,46.36
80253,"""CNR7652202""",4060.0,42.72
61674,"""CNR8875064""",4044.0,4.97


**Prompt 20: Export numeric fields for correlation matrix**

In [43]:
query = """
SELECT "Ride Distance", "Booking Value", "Driver Ratings", "Customer Rating"
FROM ncr_ride_bookings
WHERE "Ride Distance" IS NOT NULL
  AND "Booking Value" IS NOT NULL
  AND "Driver Ratings" IS NOT NULL
  AND "Customer Rating" IS NOT NULL;
"""
df_numeric = pd.read_sql(query, conn)
df_numeric.head()


,Ride Distance,Booking Value,Driver Ratings,Customer Rating
0,13.58,627.0,4.9,4.9
1,34.02,416.0,4.6,5.0
2,48.21,737.0,4.1,4.3
3,4.85,316.0,4.1,4.6
4,41.24,640.0,4.0,4.1
